In [1]:
import requests

url = "https://api.yelp.com/v3/businesses/search?sort_by=best_match&limit=20"

headers = {"accept": "_08Up8W9EhqMxywedXw1twax13yV86suQQwMcBOWZ5IflFdSRgFj1MG1kG7K2CWL9sPrkVTXZbICqPFtXGsZRqa1eTC_yTuoflTCBKVdtYVkc5JDSLREcPsvdYOKY3Yx"}

response = requests.get(url, headers=headers)

print(response.text)

{"error": {"code": "VALIDATION_ERROR", "description": "Authorization is a required parameter.", "field": "Authorization", "instance": null}}
